# Diplomatura en ciencia de datos, aprendizaje automático y sus aplicaciones - Edición 2023 - FAMAF (UNC)

## Aprendizaje no supervisado

### Trabajo práctico entregable - Grupo 22 - FIFA female players 2023 - Parte 2: implementación de modelos de ML

**Integrantes:**
- Chevallier-Boutell, Ignacio José
- Ribetto, Federico Daniel
- Rosa, Santiago
- Spano, Marcelo

**Seguimiento:** Meinardi, Vanesa

---

## Librerías

Inicializamos el entorno.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import decomposition, preprocessing
from sklearn.mixture import GaussianMixture

pd.set_option('display.max_columns', 150)
pd.set_option('display.max_rows',150)
sns.set_context('talk')
sns.set_theme(style='white')

## Lectura del dataset

Cargamos el conjunto de datos procesado previamente.

In [ ]:
path = 'fifa2023.csv'
df = pd.read_csv(path)

In [ ]:
df.head()

Como fue mencionado previamente, se decide eliminar a las arqueras de la clusterización.

Se seleccionan las variables que se utilizarán para clusterizar.

In [ ]:
df_mod = df[df['pos_gral'] != 'arq'].copy()

vars_mod = ['crossing', 'finishing', 'heading', 'short_passing', 'volleys', 
            'marking', 'standing_tackle', 'sliding_tackle', 'acceleration', 
            'sprint', 'agility', 'balance', 'shot_power', 'stamina', 
            'long_shots', 'dribbling', 'curve', 'fk_acc', 'long_passing', 
            'ball_control', 'aggression', 'interceptions', 'positioning', 
            'vision', 'penalties', 'composure', 'ls', 'st', 'rs', 'lw', 'lf', 
            'cf', 'rf', 'rw', 'lam', 'cam', 'ram', 'lm', 'lcm', 'cm', 'rcm', 
            'rm', 'ldm', 'cdm', 'rdm', 'lwb', 'rwb', 'lb', 'lcb', 'cb', 'rcb', 
            'rb']
df_mod = df_mod[vars_mod]
df_mod.shape

In [ ]:
df_mod.head()

## GMM

### Vemos el porcentaje de jugadoras en cada cluster para diferentes cantidades de componentes entre 2 y 10

Utilizamos covariance_type='full' ya que suponemos que cada cluster puede poseer posición y forma diferentes.

In [ ]:
value_counts = []
n_clusters=np.arange(2, 11)
sils=[]
sils_err=[]
for n in n_clusters:
    gmm=GaussianMixture(n, covariance_type='full', random_state=42).fit(df_mod) 
    labels=gmm.predict(df_mod)
    value_counts.append(pd.DataFrame(labels).value_counts(normalize=True).reset_index(drop=True))

In [ ]:
df_clusters = pd.DataFrame(value_counts).reset_index(drop=True)
df_clusters.insert(loc=0, column='n_components', value=np.arange(2,11))
df_clusters


Seleccionamos n=5 para así tener clusters que tengan al menos un 10% de jugadoras. Además, con 5 clusters no hay mucha variación en el porcentaje de jugadoras por cluster: el cluster más grande contiene al 27% de los jugadoras mientras que el más chico el 14%.

In [ ]:
n = 5
gmm = GaussianMixture(n, covariance_type='full', random_state=42).fit(df_mod) 
labels = gmm.predict(df_mod)
df_res = df[df['pos_gral'] != 'arq'].copy()
df_res['cluster'] = labels

In [ ]:
df_res.cluster.value_counts(normalize=True)

### Distribución de posiciones por cluster

A continuación se muestra, para cada cluster, qué porcentaje de jugadoras hay para cada posición.

* Cluster 0: En orden de porcentaje decreciente, el cluster se compone por:
    * Mediocampista defensiva central: 22.7%
    * Defensora central: 22%
    * Defensora izquierda: 19.6%
    * Defensora derecha: 19%
    * Mediocampista central: 11.6%

    El resto de las posiciones componen menos del 5% del cluster.
<br/><br/>
* Cluster 1: En orden de porcentaje decreciente, el cluster se compone por:
    * Pateadora central: 50.4%
    * Mediocampista derecha: 12.6%
    * Mediocampista ofensiva central: 12.4%
    * Volante delantera derecha: 9%
    * Mediocampista izquierda: 7%
    * Volante delantera izquierda: 7%
    * Mediocampista central: 1%
    * Delantera central: 0.5%
<br/><br/>
* Cluster 2: En orden de porcentaje decreciente, el cluster se compone por:
    * Mediocampista central: 21.9%
    * Pateadora central: 21.8%
    * Mediocampista ofensiva central: 18.1%
    * Volante delantera derecha: 10.1%
    * Mediocampista derecha: 9.8%
    * Volante delantera izquierda: 9.3%
    * Mediocampista izquierda: 8.1%
    * Defensora derecha: 1%
<br/><br/>
* Cluster 3: En orden de porcentaje decreciente, el cluster se compone por:
    * Defensora central: 58.5%
    * Defensora izquierda: 15.8%
    * Defensora derecha: 15.3%
    * Mediocampista defensiva central: 7.1%
    
    El resto de las posiciones componen menos del 5% del cluster.
<br/><br/>
* Cluster 4: En orden de porcentaje decreciente, el cluster se compone por:
    * Mediocampista central: 56.8%
    * Mediocampista defensiva central: 15.1%
    * Defensora derecha: 8%
    * Mediocampista ofensiva central: 5.9%
    * Mediocampista izquierda: 3.5%
    * Mediocampista derecha: 3.3%
    * Defensora izquierda: 2.9% 
    
    El resto de las posiciones componen menos del 5% del cluster.

In [ ]:
df_res.groupby('cluster').position.value_counts(normalize=True)

### Distribución de las variables por cluster

Del gráfico siguiente se deduce que en los clusters 0 y 2 se concentran las jugadoras con mayor overall.

In [ ]:
sns.boxplot(x='cluster', y='overall', data=df_res)
plt.show()

A continuación se grafica la distribución por cluster para todas las variables que se utilizaron para clusterizar con lo cual se puede caracterizar cada cluster:

* Cluster 0: Posee los valores de ldm, cdm, rdm, lwb, cwb, rwb, lb, lcb, cb, rcb, rb más altos. Estas variables representan el puntaje asignado a posiciones de defensa.
* Cluster 1: Posee los valores más bajos de marking, standing tackle, siding tackle, aggression, interceptions, ldm, cdm, rdm, lwb, cwb, rwb, lb, lcb, cb, rcb, rb.
* Cluster 2: Posee los valores más altos de finishing, volleys, acceleration, sprint, agility, shot power, long shots, dribblign, curve, fk_acc, ball_control, positioning, vision, penalties, ls, st, rs, lw, lf, cf, rf, rw, lam, cam, ram, lm, rm. Estas variables representan el puntaje asignado a características ofensivas, movimientos, potencia, habilidades físicas, y posiciones de mediacompista, delandera, volante y pateadora.
* Cluster 3: Posee los valores más bajos de finishing, volleys, shot_power, long_shots, dribbling, curve, fk_acc, ball_control, positioning, vision, penalties, ls, st, rs, lw, lf, cf, rf, rw, lam, cam, ram, lm, lcm, cm, rcm, rm.
* Cluster 4: Posee valores intermedios en todas sus variables. 

In [ ]:
for col in vars_mod:
    sns.boxplot(x='cluster', y=col, data=df_res)
    plt.show()

# PCA

Para visualizar los resultados de la clusterización se realiza un embedding con PCA.

In [ ]:
std_scale=preprocessing.StandardScaler().fit(df_mod)
X_scaled=std_scale.transform(df_mod)

A continuación se realiza una descomposición en las 3 componentes principales utilizando PCA y se observa la varianza para cada componente y la acumulada. Con las primeras dos componentes se tiene el 79% de la varianza, y la tercer componente solo aporta un 4% más.

In [ ]:
pca=decomposition.PCA(n_components=3)

pca.fit(X_scaled) #input data is centered but not scaled for each feature before applying the SVD

# proporción de varianza
print('proporción de varianza por componente: ', pca.explained_variance_ratio_)
# proporción de varianza acumulada
print ('proporción de varianza por componente acumulada: ', pca.explained_variance_ratio_.cumsum())

X_projected=pca.transform(X_scaled) #numpy array
print ('tamaño de los datos: ', X_projected.shape)

In [ ]:
df_projected = pd.DataFrame(X_projected)
df_projected['cluster'] = labels

Se grafican las primeras dos componentes y se colorean los diferentes clusters.
Se puede ver una superposición entre los clusters 1 y 2 y entre los clusters 0 y 3.

In [ ]:
plt.figure(figsize=(20,10))
sns.scatterplot(data=df_projected, x=0, y=1, hue='cluster', palette='tab10')
plt.grid()
plt.show()

## Conclusiones

Se realizó un clustering utilizando Gaussian Mixture Models (GMM). Se probaron diferentes cantidades de componentes y se definió utilizar 5 componentes para tener clusters que tengan al menos un 10% de jugadoras. Además, con 5 clusters no hay tanta variación en el porcentaje de jugadoras por cluster: el cluster más grande contiene al 27% de los jugadoras mientras que el más chico el 14%.

Del análisis realizado sobre los clusters se puede concluir lo siguiente:
* El cluster 0 está compuesto por defensoras y mediocampistas centrales y poseen el puntaje asignado a posiciones de defensa más altos. 
* El cluster 1 está compuesto mayoritariamente por pateadoras centrales, componiendo el 50% del cluster. El resto son mediocampistas y volantes. Estas jugadoras poseen bajos valores en los puntajes asignados a posiciones de defensa.
* El cluster 2 está compuesto por pateadoras centrales, mediocampistas y volantes. La diferencia con el cluster 1 es que posee los puntajes asignados más altos a características ofensivas, movimientos, potencia, habilidades físicas, y posiciones de mediacompista, delandera, volante y pateadora.
* Por lo visto en los puntos anteriores, los clusters 1 y 2 poseen jugadoras que juegan en las mismas posiciones aunque con habilidades diferentes. Y por esta razón en el gráfico de PCA se observa una superposición entre estos clusters.
* El cluster 3 se encuentra conformado por defensoras. La diferencia con el cluster 0 es que posee los puntajes asignados más bajos a habilidades físicas y posiciones de mediacompista, delandera, volante y pateadora.
* Los clusters 0 y 3 están conformados mayoritariamente por defensoras pero con puntajes diferentes. Por esta razón en el gráfico de pCA se observa una superposición entre estos clusters.
* El cluster 4 posee mayoritariamente mediocampistas centrales (un 57%). Además se compone de otras mediocampistas y defensoras. Posee valores intermedios en todas las variables, lo cual es observable en el gráfico de PCA, en donde se encuentra en el medio del gráfico y rodeado por el resto de los clusters.
* Como en cada cluster los puntajes de las características son similares entre si, se podría considerar intercambiar jugadoras que se encuentran en un mismo cluster de ser necesario. Además, podría considerarse al cluster 4 como un cluster comodín ya que las jugadoras pertenecientes al mismo poseen habilidades y puntajes intermedios y juegan en diferentes posiciones.